Things to remember:
1. Separate memory space
2. Multiple cores of the cpu
3. Async allowed

### 1_basic_multi.py

```(python, eval=False)
def worker_function(name):
    do something;

processes = []
    for i in range(3):
        p = multiprocessing.Process(target=worker_function, args=(f"Worker-{i}",))
        processes.append(p)
        p.start()
    for p in processes:
        p.join()
```
- create process. append in a list.
- start the process.
- join.

### 2_basic_multi_q.py

Control the process through a queue.
```(python, eval= False)
def worker_with_result(name, queue):
    result = 
    queue.put(result)

result_queue = multiprocessing.Queue() 
    # Create and start processes
    processes = []
    for i in range(3):
        p = multiprocessing.Process(target=worker_with_result, 
                                    args=(f"Process-{i}", result_queue))
        processes.append(p)
        p.start()
    for p in processes:
        p.join()
    results = []
    while not result_queue.empty():
        results.append(result_queue.get())
```
- start a queue.
- Put the result in the queue while processing.
- start and join the process.
- retrive the results with result_queue.


### 3_basic_seq_par.py
A simple idea to check for sequential and parallel processing.

```(python, eval =False)
with multiprocessing.Pool(processes=4) as pool:
    results = pool.map(cpu_intensive_task, [1000000] * 4)
```
Result is
Sequential execution:
Sequential time: 0.11 seconds

Parallel execution:
Parallel time: 0.08 seconds

### 4_basic_pool.py

Different types of multiProcessing.pool

```(python, eval = False)

with multiprocessing.Pool(processes=4) as pool:
    squares = pool.map(square_number, numbers)
    pairs = [(2, 3), (4, 5), (6, 7)]
    powers = pool.starmap(pow, pairs)
    async_results = []
    for num in numbers[:5]:
        result = pool.apply_async(square_number, (num,))
        async_results.append(result)
    async_squares = [result.get() for result in async_results]
```

### 5_adv_pipe.py
Inter Process communication

```(python, eval=False)
def sender(conn, data):
    for item in data:
        conn.send(item)
        time.sleep(0.1)
    conn.send("DONE")
    conn.close()
def receiver(conn):
    while True:
        msg = conn.recv()
        if msg == "DONE":
            break
    conn.close()


parent_conn, child_conn = multiprocessing.Pipe()
data = ["Hello", "World", "From", "Multiprocessing"]
sender_process = multiprocessing.Process(target=sender, args=(parent_conn, data))
receiver_process = multiprocessing.Process(target=receiver, args=(child_conn,))
sender_process.start()
receiver_process.start()
sender_process.join()
receiver_process.join()
```


### 6_adv_shared_mem.py
This is way to handle shared memory.

```(python, eval=False)


shared_value = multiprocessing.Value('i', 0)  # 'i' for integer
shared_array = multiprocessing.Array('i', [0] * 4)  # Array of 4 integers
lock = multiprocessing.Lock()
processes = []
for i in range(4):
    p = multiprocessing.Process(target=worker_with_shared_memory,
                                args=(shared_value, shared_array, lock, i))
```
In this way, with lock the memory is updated 

```(python, eval = False)

with lock:
    shared_value.value += 1
    shared_array[worker_id] = shared_value.value
```

### 7_adv_prod_consumer.py

```(python, eval=False)
def producer(queue, num_items):
    for i in range(num_items):
        item = f"item_{i}"
        queue.put(item)
        time.sleep(0.1)
    queue.put(None)
def consumer(queue, consumer_id):
    while True:
        item = queue.get()
        if item is None:
            queue.put(None)
            break
        print(f"Consumer {consumer_id} processing: {item}")
        time.sleep(0.2)  

queue = multiprocessing.Queue()
producer_proc = multiprocessing.Process(target=producer, args=(queue, 10))
consumer_procs = []
for i in range(3):
    p = multiprocessing.Process(target=consumer, args=(queue, i))
    consumer_procs.append(p)
producer_proc.start()
for p in consumer_procs:
    p.start()
producer_proc.join()
for p in consumer_procs:
    p.join()

```

### 8_adv_manager.py

A manager simplifies inter-process communication and shared data management by providing a centralized, process-safe mechanism for accessing and manipulating objects across multiple processes.

```(python, eval= False)
def worker_with_manager(shared_dict, shared_list, worker_id):
    shared_dict[f"worker_{worker_id}"] = f"Hello from worker {worker_id}"
    shared_list.append(f"Item from worker {worker_id}")
    time.sleep(0.1)

manager = multiprocessing.Manager()
shared_dict = manager.dict()
shared_list = manager.list()
processes = []
for i in range(4):
    p = multiprocessing.Process(target=worker_with_manager,
                                args=(shared_dict, shared_list, i))
    processes.append(p)
    p.start()
```
